### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import aomap
from survey_tools import healpix
from survey_tools.utility import plot

### Options

In [ ]:
config = aomap.read_config('config.yaml')
ao_system_name = 'GNAO-Optimal'

object_name = 'cosmos'
match object_name:
    case 'cosmos':
        ra = 150.0
        dec = 2.0
    case 'omegacentauri':
        ra = 201.05
        dec = -47.48
    case 'galcenter':
        ra = 266.42
        dec = -29.00
    case 'andromeda':
        ra = 10.68
        dec = 41.27
    case 'lmc':
        ra = 80.89
        dec = -69.76 + 8
    case 'smc':
        ra = 13.19
        dec = -72.83
    case 'm13':
        ra = 250.4
        dec = 36.46

outer_pix = healpix.get_healpix_from_skycoord(config.outer_level, SkyCoord(ra=ra*u.deg, dec=dec*u.deg))
print(f"Outer pixel: {outer_pix}")

### Steller Density Limit

In [ ]:
aomap.plot_map(aomap.get_map_data(config, 9, 'star-density'),
               milkyway=True, milkyway_width=20,
               surveys=[['ews', {'edgecolor': 'red', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-north', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-south', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-fornax', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}]],
               points=[[ra, dec, object_name.upper()], {'marker': 's', 's': 25, 'edgecolor': 'white', 'facecolor': 'none'}],
               rotation=110, projection='cart', width=15,
               norm='linear', vmin=None, vmax=config.asterisms_max_star_density
)

### Find Asterisms

In [ ]:
asterisms, stars, ngs, ao_system, overlap_level = aomap.find_outer_asterisms(config, outer_pix, ao_system_name, return_detail=True, use_cache=True, verbose=True)

num_stars = len(stars)
print(f"Num Stars    : {num_stars}")
num_ngs = len(ngs)
print(f"Num NGS      : {num_ngs}")
num_asterisms = len(asterisms)
print(f"Num Asterisms: {num_asterisms}")

### Plot

In [ ]:
fig, ax = plot.create_plot(title=f"GAIA Stars")
values, bins, _ = ax.hist(stars[ao_system['band']], min(30,int(np.sqrt(num_stars))), rwidth=0.9, color='blue')
plot.format_hist(ax, values=values, bins=bins, xtickspacing=2, xforceinteger=True, ytickspacing=100)
ax.set_xlabel('R')
ax.set_ylabel('Stars')
ax.axvline(16.0, linestyle='--', color='k')
ax.text(16.0+0.1, ax.get_ylim()[1] * 0.95, 'Optimal', rotation=90, verticalalignment='top', color='k')
ax.axvline(17.0, linestyle='--', color='k')
ax.text(17.0+0.1, ax.get_ylim()[1] * 0.95, 'Nominal', rotation=90, verticalalignment='top', color='k')
ax.axvline(18.5, linestyle='--', color='k')
ax.text(18.5+0.1, ax.get_ylim()[1] * 0.95, 'Limit', rotation=90, verticalalignment='top', color='k')
plt.show()

In [ ]:
zoom_levels = 0
zoom_pix_offset = np.array([0,1,2,3]) if zoom_levels > 0 else 0
plot_type = 'dust' # dust or stars

level = config.outer_level + zoom_levels
pixs = outer_pix * 4**(level - config.outer_level) + zoom_pix_offset

aomap.plot_map(aomap.get_map_data(config, (config.max_data_level if plot_type == 'dust' else config.inner_level), ('dust-extinction' if plot_type == 'dust' else 'star-density'), level=level, pixs=pixs),
    stars=[stars if len(stars) < 10000 else ngs, {'band': ao_system['band'], 'min_mag': 16, 'max_mag': 19, 'max_size': 50, 'fc': 'white', 'ec': 'black'}],
    asterisms=[asterisms, {'fov': ao_system['fov'], 'fov_1ngs': ao_system['fov_1ngs']}],
    projection='astro', norm='linear', width=12, boundaries_level=config.outer_level, boundaries_pixs=0, cbar_format=('%.3f' if plot_type == 'dust' else '%d')
)